In [3]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from helper import *

In [4]:
file_path_HLES = '../Dataset/SurveyData/SurveyData_CSV_DAP_2021_HLES_dog_owner_v1.0.csv'
df_HLES = pd.read_csv(file_path_HLES)
df_HLES

/var/folders/zd/3_6frj8s08b9zb1f3y0xqr2r0000gn/T/ipykernel_94783/1766648619.py:2: DtypeWarning: Columns (5,21,75,77,79,81,83,85,87,89,191,192,193,194,195,196,197,198,199,200,201,204,214,285,295,516,566) have mixed types. Specify dtype option on import or set low_memory=False.
  df_HLES = pd.read_csv(file_path_HLES)


,dog_id,owner_id,st_hles_completion_date,dd_breed_pure_or_mixed,dd_breed_pure,dd_breed_pure_non_akc,dd_breed_mixed_primary,dd_breed_mixed_secondary,dd_age_years,dd_age_basis,...,oc_primary_residence_state,oc_primary_residence_census_division,oc_primary_residence_ownership,oc_primary_residence_ownership_other_description,oc_primary_residence_time_percentage,oc_secondary_residence,oc_secondary_residence_state,oc_secondary_residence_ownership,oc_secondary_residence_ownership_other_description,oc_secondary_residence_time_percentage
0,10509,14660,2020-10-17,2,NaN,NaN,16.0,999.0,16.333333,2,...,MO,4.0,2,NaN,NaN,False,NaN,NaN,NaN,NaN
1,74227,16755,2020-10-26,2,NaN,NaN,133.0,12.0,1.833333,1,...,MD,5.0,2,NaN,NaN,False,NaN,NaN,NaN,NaN
2,32487,8568,2020-09-07,1,159.0,NaN,NaN,NaN,13.000000,3,...,VA,5.0,1,NaN,NaN,False,NaN,NaN,NaN,NaN
3,63150,5380,2020-08-21,2,NaN,NaN,999.0,999.0,13.000000,3,...,CA,9.0,2,NaN,NaN,False,NaN,NaN,NaN,NaN
4,33369,15957,2020-10-23,2,NaN,NaN,230.0,82.0,2.166667,1,...,OH,3.0,2,NaN,NaN,False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33167,91849,32736,2021-06-22,2,NaN,NaN,276.0,1.0,10.000000,3,...,CA,9.0,2,NaN,5.0,True,NH,2.0,NaN,1.0
33168,33783,9482,2020-09-15,2,NaN,NaN,130.0,195.0,3.000000,3,...,WA,9.0,1,NaN,NaN,False,NaN,NaN,NaN,NaN
33169,98862,32312,2021-05-25,2,NaN,NaN,60.0,49.0,4.000000,3,...,AR,7.0,2,NaN,NaN,False,NaN,NaN,NaN,NaN
33170,99462,32692,2021-06-18,2,NaN,NaN,204.0,27.0,0.916667,1,...,WA,9.0,2,NaN,NaN,False,NaN,NaN,NaN,NaN


In [148]:
# Map the classification dict in helper.py to the 'Breed' column
df_HLES['dd_breed_mixed_primary'] = df_HLES['dd_breed_mixed_primary'].map(lambda x: pure_breed_mapping.get(x, x))
df_HLES['dd_breed_mixed_secondary'] = df_HLES['dd_breed_mixed_secondary'].map(lambda x: pure_breed_mapping.get(x, x))
df_HLES['dd_breed_pure'] = df_HLES['dd_breed_pure'].map(lambda x: pure_breed_mapping.get(x, x))
df_HLES[['dog_id','dd_breed_pure_non_akc', 'dd_breed_pure' ,'dd_breed_mixed_primary', 'dd_breed_mixed_secondary']]

,dog_id,dd_breed_pure_non_akc,dd_breed_pure,dd_breed_mixed_primary,dd_breed_mixed_secondary
0,10509,NaN,NaN,Australian Cattle Dog,Unknown
1,74227,NaN,NaN,Ibizan Hound,American Staffordshire Terrier
2,32487,NaN,Lhasa Apso,NaN,NaN
3,63150,NaN,NaN,Unknown,Unknown
4,33369,NaN,NaN,Siberian Husky,Collie
...,...,...,...,...,...
33167,91849,NaN,NaN,Yorkshire Terrier,Affenpinscher
33168,33783,NaN,NaN,Havanese,Poodle
33169,98862,NaN,NaN,Bull Terrier,Boston Terrier
33170,99462,NaN,NaN,Pug,Beagle


In [149]:
df_HLES['dd_breed'] = df_HLES['dd_breed_pure'].fillna(df_HLES['dd_breed_mixed_primary'])

# Add "Mix" to the values filled from breed_mixed_primary
df_HLES['dd_breed'] = df_HLES.apply(lambda row: f"{row['dd_breed']} Mix" if pd.notna(row['dd_breed_mixed_primary']) else row['dd_breed'], axis=1)

In [150]:
def determine_breed(row):
    if row['dd_breed_mixed_primary'] != 'Unknown':
        return f"{row['dd_breed_mixed_primary']} Mix"
    elif row['dd_breed_mixed_primary'] == 'Unknown' and row['dd_breed_mixed_secondary'] != 'Unknown':
        return f"{row['dd_breed_mixed_secondary']} Mix"
    elif row['dd_breed_mixed_primary'] == 'Unknown' and row['dd_breed_mixed_secondary'] == 'Unknown':
        return "Unknown Mix"
    else:
        pass

# Apply the function for the mixed breed and map it to the 'dd_breed' column
df_HLES['dd_breed'][df_HLES['dd_breed_pure_or_mixed']==2] = df_HLES[df_HLES['dd_breed_pure_or_mixed']==2].apply(determine_breed, axis=1)

/var/folders/zd/3_6frj8s08b9zb1f3y0xqr2r0000gn/T/ipykernel_61691/2592567024.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_HLES['dd_breed'][df_HLES['dd_breed_pure_or_mixed']==2] = df_HLES[df_HLES['dd_breed_pure_or_mixed']==2].apply(determine_breed, axis=1)


In [151]:
df_HLES[['dog_id', 'dd_breed', 'dd_breed_pure', 'dd_breed_mixed_primary', 'dd_breed_mixed_secondary', 'dd_breed_pure_or_mixed']]

,dog_id,dd_breed,dd_breed_pure,dd_breed_mixed_primary,dd_breed_mixed_secondary,dd_breed_pure_or_mixed
0,10509,Australian Cattle Dog Mix,NaN,Australian Cattle Dog,Unknown,2
1,74227,Ibizan Hound Mix,NaN,Ibizan Hound,American Staffordshire Terrier,2
2,32487,Lhasa Apso,Lhasa Apso,NaN,NaN,1
3,63150,Unknown Mix,NaN,Unknown,Unknown,2
4,33369,Siberian Husky Mix,NaN,Siberian Husky,Collie,2
...,...,...,...,...,...,...
33167,91849,Yorkshire Terrier Mix,NaN,Yorkshire Terrier,Affenpinscher,2
33168,33783,Havanese Mix,NaN,Havanese,Poodle,2
33169,98862,Bull Terrier Mix,NaN,Bull Terrier,Boston Terrier,2
33170,99462,Pug Mix,NaN,Pug,Beagle,2


In [152]:
def classify_breed(weight_class):
    if weight_class in [1, 2]:
        return 'Mixed Breed Small (0 - 20lbs)'
    elif weight_class in [3, 4, 5, 6]:
        return 'Mixed Breed Medium (21 - 60lbs)'
    elif weight_class in [7, 8, 9, 10, 11]:
        return 'Mixed Breed Large (61lbs+)'
    else:
        return 'Unknown Mix'

# Apply the classification function only to rows where "Breed" contains "Unknown/Unknown"
mask = df_HLES['dd_breed'].str.contains("Unknown Mix")
df_HLES.loc[mask, 'dd_breed'] = df_HLES.loc[mask, 'dd_weight_range'].apply(classify_breed)

In [153]:
df_HLES.loc[
    (df_HLES['dd_breed_mixed_primary'] == "Unknown") & (df_HLES['dd_breed_mixed_secondary'] == "Unknown"),
    ['dd_breed', 'dd_breed_mixed_primary', 'dd_breed_mixed_secondary']
]

,dd_breed,dd_breed_mixed_primary,dd_breed_mixed_secondary
3,Mixed Breed Large (61lbs+),Unknown,Unknown
66,Mixed Breed Medium (21 - 60lbs),Unknown,Unknown
260,Mixed Breed Large (61lbs+),Unknown,Unknown
294,Mixed Breed Medium (21 - 60lbs),Unknown,Unknown
310,Mixed Breed Medium (21 - 60lbs),Unknown,Unknown
...,...,...,...
32874,Mixed Breed Medium (21 - 60lbs),Unknown,Unknown
32888,Mixed Breed Medium (21 - 60lbs),Unknown,Unknown
33100,Mixed Breed Medium (21 - 60lbs),Unknown,Unknown
33119,Mixed Breed Medium (21 - 60lbs),Unknown,Unknown


In [154]:
df_HLES['dd_breed'][df_HLES['dd_breed'].str.contains('Mixed')].value_counts()

dd_breed
Mixed Breed Medium (21 - 60lbs)    335
Mixed Breed Large (61lbs+)          76
Mixed Breed Small (0 - 20lbs)       64
Name: count, dtype: int64

In [155]:
df_HLES[['dd_breed', 'dd_breed_mixed_primary', 'dd_breed_mixed_secondary']][df_HLES['dd_breed_pure_non_akc'].notna()]

,dd_breed,dd_breed_mixed_primary,dd_breed_mixed_secondary
1109,My dog is a non-AKC purebred not listed here,NaN,NaN
1213,My dog is a non-AKC purebred not listed here,NaN,NaN
1739,My dog is a non-AKC purebred not listed here,NaN,NaN
2487,My dog is a non-AKC purebred not listed here,NaN,NaN
2603,My dog is a non-AKC purebred not listed here,NaN,NaN
...,...,...,...
31092,My dog is a non-AKC purebred not listed here,NaN,NaN
31099,My dog is a non-AKC purebred not listed here,NaN,NaN
31243,My dog is a non-AKC purebred not listed here,NaN,NaN
32340,My dog is a non-AKC purebred not listed here,NaN,NaN


In [156]:
df_HLES['dd_breed_pure_non_akc'].value_counts()

dd_breed_pure_non_akc
Alaskan Klee Kai                              5
English Shepherd                              4
Silken Windhound                              4
American Bully                                4
Shiloh Shepherd                               3
Maremma Sheepdog                              3
American Bulldog                              2
Berger Blanc Suisse / White Swiss Shepherd    2
Japanese Spitz                                2
Australian Labradoddle                        2
Australian Labradoodle                        2
Miniature Australian Shepherd                 2
Austrailian Labradoodle                       1
Akbash                                        1
Japanese Akita Inu                            1
PODENCO                                       1
Labradoodle                                   1
labradoodle                                   1
Mini Australian Shepherd                      1
Large Munsterlander                           1
Miki              

In [157]:
df_HLES['dd_breed'].value_counts()

dd_breed
Labrador Retriever Mix     2345
Labrador Retriever         1985
Golden Retriever           1728
Poodle Mix                  855
German Shepherd Dog Mix     815
                           ... 
Norwegian Buhund Mix          1
Hovawart Mix                  1
Swedish Vallhund              1
Beauceron Mix                 1
Norwegian Lundhund            1
Name: count, Length: 462, dtype: int64

In [158]:
condition = df_HLES['dd_breed'] == "My dog is a non-AKC purebred not listed here"
df_HLES.loc[condition, 'dd_breed'] = df_HLES.loc[condition, 'dd_breed_pure_non_akc'].values
df_HLES[['dd_breed', 'dd_breed_mixed_primary', 'dd_breed_mixed_secondary']][df_HLES['dd_breed_pure_non_akc'].notna()]

,dd_breed,dd_breed_mixed_primary,dd_breed_mixed_secondary
1109,Mini Australian Shepherd,NaN,NaN
1213,Miniature Australian Shepherd,NaN,NaN
1739,Miniature Poodle,NaN,NaN
2487,Shiloh Shepherd,NaN,NaN
2603,Windsprite,NaN,NaN
...,...,...,...
31092,Prague Ratter,NaN,NaN
31099,Toy Austrailan Shepherd,NaN,NaN
31243,Australian Labradoodle,NaN,NaN
32340,Miniature Australian Shepherd,NaN,NaN


In [159]:
df_HLES[['dd_breed']][df_HLES['dd_breed_pure_non_akc'].notna()]

,dd_breed
1109,Mini Australian Shepherd
1213,Miniature Australian Shepherd
1739,Miniature Poodle
2487,Shiloh Shepherd
2603,Windsprite
...,...
31092,Prague Ratter
31099,Toy Austrailan Shepherd
31243,Australian Labradoodle
32340,Miniature Australian Shepherd


In [160]:
# Map the classification_2 dict in helper.py to the 'dd_breed' column
df_HLES['dd_breed'] = df_HLES['dd_breed'].map(lambda x: classification_mapping_2.get(x, x))
df_HLES['dd_breed']

0           Australian Cattle Dog Mix
1                    Ibizan Hound Mix
2                          Lhasa Apso
3          Mixed Breed Large (61lbs+)
4                  Siberian Husky Mix
                     ...             
33167           Yorkshire Terrier Mix
33168                    Havanese Mix
33169                Bull Terrier Mix
33170                         Pug Mix
33171    Treeing Walker Coonhound Mix
Name: dd_breed, Length: 33172, dtype: object

## Merge with Breed_grouping_HLES_extended on dd_breed column

In [161]:
file_path_breeds = "../Dataset/Supplementary_data/breed_grouping_HLES_extended.csv"
df_breeds = pd.read_csv(file_path_breeds, delimiter=";", encoding="ISO-8859-1")
df_breeds

,Breed Type,Sub Breed,Breed Group,dd_breed
0,Ancient and Spitz,Ancient and Spitz,Ancient and Spitz,Ainu Dog
1,Ancient and Spitz,Ancient and Spitz,Ancient and Spitz,Ainu Dog Mix
2,Ancient and Spitz,Ancient and Spitz,Ancient and Spitz,Akita
3,Ancient and Spitz,Ancient and Spitz,Ancient and Spitz,Akita Mix
4,Ancient and Spitz,Ancient and Spitz,Ancient and Spitz,Alaskan Husky
...,...,...,...,...
954,Working dogs,Shepherd - Other,Shepherd,Shiloh Shepherd Mix
955,Working dogs,Shepherd - Other,Shepherd,shiloh shepherd
956,Working dogs,Shepherd - Other,Shepherd,shiloh shepherd Mix
957,Working dogs,Shepherd - Other,Shepherd,Tervuren


In [162]:
def map_breed_info(row):
    breed_type, sub_breed, breed_group = None, None, None

    # Check if 'dd_breed' is in the df_breeds DataFrame
    if row['dd_breed'] in df_breeds['dd_breed'].values:
        breed_type = df_breeds.loc[df_breeds['dd_breed'] == row['dd_breed'], 'Breed Type'].iloc[0]
        sub_breed = df_breeds.loc[df_breeds['dd_breed'] == row['dd_breed'], 'Sub Breed'].iloc[0]
        breed_group = df_breeds.loc[df_breeds['dd_breed'] == row['dd_breed'], 'Breed Group'].iloc[0]

    primary_breed_type, primary_sub_breed, primary_breed_group = None, None, None
    # Check if 'dd_breed_mixed_primary' is in the df_breeds DataFrame
    if row['dd_breed_mixed_primary'] in df_breeds['dd_breed'].values:
        primary_breed_type = df_breeds.loc[df_breeds['dd_breed'] == row['dd_breed_mixed_primary'], 'Breed Type'].iloc[0]
        primary_sub_breed = df_breeds.loc[df_breeds['dd_breed'] == row['dd_breed_mixed_primary'], 'Sub Breed'].iloc[0]
        primary_breed_group = df_breeds.loc[df_breeds['dd_breed'] == row['dd_breed_mixed_primary'], 'Breed Group'].iloc[0]

    secondary_breed_type, secondary_sub_breed, secondary_breed_group = None, None, None
    # Check if 'dd_breed_mixed_secondary' is in the df_breeds DataFrame
    if row['dd_breed_mixed_secondary'] in df_breeds['dd_breed'].values:
        secondary_breed_type = df_breeds.loc[df_breeds['dd_breed'] == row['dd_breed_mixed_secondary'], 'Breed Type'].iloc[0]
        secondary_sub_breed = df_breeds.loc[df_breeds['dd_breed'] == row['dd_breed_mixed_secondary'], 'Sub Breed'].iloc[0]
        secondary_breed_group = df_breeds.loc[df_breeds['dd_breed'] == row['dd_breed_mixed_secondary'], 'Breed Group'].iloc[0]

    return pd.Series({
        'Breed Type': breed_type,
        'Sub Breed': sub_breed,
        'Breed Group': breed_group,
        'Primary Breed Type': primary_breed_type,
        'Primary Sub Breed': primary_sub_breed,
        'Primary Breed Group': primary_breed_group,
        'Secondary Breed Type': secondary_breed_type,
        'Secondary Sub Breed': secondary_sub_breed,
        'Secondary Breed Group': secondary_breed_group
    })

# Apply the function to create new columns in df_HLES
df_HLES[['Breed Type', 'Sub Breed', 'Breed Group',
         'Primary Breed Type', 'Primary Sub Breed', 'Primary Breed Group',
         'Secondary Breed Type', 'Secondary Sub Breed', 'Secondary Breed Group']] = df_HLES.apply(map_breed_info, axis=1)

In [163]:
df_HLES[['dd_breed', 'dd_breed_pure', 'dd_breed_mixed_primary', 'dd_breed_mixed_secondary', 'Breed Group', 'Primary Breed Type', 'Primary Sub Breed', 'Primary Breed Group',
         'Secondary Breed Type', 'Secondary Sub Breed', 'Secondary Breed Group']][(df_HLES['dd_breed_pure_non_akc'].notna()) | (df_HLES['dd_breed'] == 'Goldendoodle')]

,dd_breed,dd_breed_pure,dd_breed_mixed_primary,dd_breed_mixed_secondary,Breed Group,Primary Breed Type,Primary Sub Breed,Primary Breed Group,Secondary Breed Type,Secondary Sub Breed,Secondary Breed Group
1109,Miniature American Shepherd,My dog is a non-AKC purebred not listed here,NaN,NaN,Australian-like,None,None,None,None,None,None
1213,Miniature American Shepherd,My dog is a non-AKC purebred not listed here,NaN,NaN,Australian-like,None,None,None,None,None,None
1739,Miniature Poodle,My dog is a non-AKC purebred not listed here,NaN,NaN,Working dogs - Non-sport,None,None,None,None,None,None
2487,Shiloh Shepherd,My dog is a non-AKC purebred not listed here,NaN,NaN,Shepherd,None,None,None,None,None,None
2603,Windsprite,My dog is a non-AKC purebred not listed here,NaN,NaN,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
31092,Prague Ratter,My dog is a non-AKC purebred not listed here,NaN,NaN,None,None,None,None,None,None,None
31099,Toy Austrailan Shepherd,My dog is a non-AKC purebred not listed here,NaN,NaN,None,None,None,None,None,None,None
31243,Australian Labradoodle,My dog is a non-AKC purebred not listed here,NaN,NaN,None,None,None,None,None,None,None
32340,Miniature American Shepherd,My dog is a non-AKC purebred not listed here,NaN,NaN,Australian-like,None,None,None,None,None,None


In [164]:
df_HLES['Breed Group'].isna().sum()

69

In [165]:
df_HLES[['dog_id', 'dd_breed_pure', 'dd_breed_mixed_primary', 'dd_breed_mixed_secondary', 'Breed Group', 'Primary Breed Group', 'Secondary Breed Group']][(df_HLES['Primary Breed Group'] == df_HLES['Secondary Breed Group']) & (df_HLES['Primary Breed Group'] != 'None') & (df_HLES['Primary Breed Group'] != df_HLES['Breed Group'])]

,dog_id,dd_breed_pure,dd_breed_mixed_primary,dd_breed_mixed_secondary,Breed Group,Primary Breed Group,Secondary Breed Group
115,45805,NaN,Boxer,Great Pyrenees,Mixed Other,Mastiff-like Group 1,Mastiff-like Group 1
1583,99881,NaN,Boxer,American Pitbull Terrier,Mixed Other,Mastiff-like Group 1,Mastiff-like Group 1
1778,54674,NaN,Boxer,Bulldog,Mixed Other,Mastiff-like Group 1,Mastiff-like Group 1
1819,52969,NaN,Labrador Retriever,Labrador Retriever,Mixed Other,Labs,Labs
1879,100920,NaN,Boxer,Bull Terrier,Mixed Other,Mastiff-like Group 1,Mastiff-like Group 1
...,...,...,...,...,...,...,...
31976,96520,NaN,Boxer,American Pitbull Terrier,Mixed Other,Mastiff-like Group 1,Mastiff-like Group 1
32019,85215,NaN,Boxer,American Staffordshire Terrier,Mixed Other,Mastiff-like Group 1,Mastiff-like Group 1
32396,12074,NaN,Boxer,American Pitbull Terrier,Mixed Other,Mastiff-like Group 1,Mastiff-like Group 1
32592,69234,NaN,Boxer,American Pitbull Terrier,Mixed Other,Mastiff-like Group 1,Mastiff-like Group 1


## Set Breed Group, Breed Type and Sub Breed if the corresponding primary and secondary values are the same

In [166]:
# Breed Group
condition = (df_HLES['Primary Breed Group'] == df_HLES['Secondary Breed Group']) & \
            (df_HLES['Primary Breed Group'] != 'None') & \
            (df_HLES['Primary Breed Group'] != df_HLES['Breed Group'])

# Copy 'Primary Breed Group' to 'Breed Group' where the condition is met
df_HLES.loc[condition, 'Breed Group'] = df_HLES.loc[condition, 'Primary Breed Group']

In [167]:
# Breed Type
condition = (df_HLES['Primary Breed Type'] == df_HLES['Secondary Breed Type']) & \
            (df_HLES['Primary Breed Type'] != 'None') & \
            (df_HLES['Primary Breed Type'] != df_HLES['Breed Type'])

# Copy 'Primary Breed Group' to 'Breed Group' where the condition is met
df_HLES.loc[condition, 'Breed Type'] = df_HLES.loc[condition, 'Primary Breed Type']

In [168]:
# Sub Breed
condition = (df_HLES['Primary Sub Breed'] == df_HLES['Secondary Sub Breed']) & \
            (df_HLES['Primary Sub Breed'] != 'None') & \
            (df_HLES['Primary Sub Breed'] != df_HLES['Sub Breed'])

# Copy 'Primary Breed Group' to 'Breed Group' where the condition is met
df_HLES.loc[condition, 'Sub Breed'] = df_HLES.loc[condition, 'Primary Sub Breed']

In [169]:
df_HLES['Breed Group'].isna().sum()

69

In [170]:
df_HLES[['dog_id', 'dd_breed_pure', 'dd_breed_mixed_primary', 'dd_breed_mixed_secondary', 'Breed Group', 'Primary Breed Group', 'Secondary Breed Group']]

,dog_id,dd_breed_pure,dd_breed_mixed_primary,dd_breed_mixed_secondary,Breed Group,Primary Breed Group,Secondary Breed Group
0,10509,NaN,Australian Cattle Dog,Unknown,Australian-like,Australian-like,None
1,74227,NaN,Ibizan Hound,American Staffordshire Terrier,Herding dogs - Other,Herding dogs - Other,Mastiff-like Group 1
2,32487,Lhasa Apso,NaN,NaN,Toy - Other,None,None
3,63150,NaN,Unknown,Unknown,Mixed Large,None,None
4,33369,NaN,Siberian Husky,Collie,Ancient and Spitz,Ancient and Spitz,Australian-like
...,...,...,...,...,...,...,...
33167,91849,NaN,Yorkshire Terrier,Affenpinscher,Terriers,Terriers,Terriers
33168,33783,NaN,Havanese,Poodle,Toy - Other,Toy - Other,Working dogs - Non-sport
33169,98862,NaN,Bull Terrier,Boston Terrier,Mastiff-like Group 1,Mastiff-like Group 1,Mastiff-like Group 1
33170,99462,NaN,Pug,Beagle,Toy - Other,Toy - Other,Hound


In [171]:
df_HLES['dd_breed'].value_counts()

dd_breed
Labrador Mix                        2345
Labrador Retriever                  1985
Golden Retriever                    1728
Poodle Mix                           855
German Shepherd Mix                  815
                                    ... 
Czechoslovakian Vlack                  1
English Foxhound                       1
Slovakian Wirehaired Pointer Mix       1
Portuguese Podengo                     1
Norwegian Lundhund                     1
Name: count, Length: 500, dtype: int64

In [172]:
df_HLES['dd_breed'][df_HLES['Breed Group'].isna()].value_counts()

dd_breed
Village Dog Mix                             16
Australian Labradoodle                       5
Wolf-hybrid Mix                              5
Jagdterrier                                  2
Drentsche Patrijshond Mix                    2
Lowchen                                      2
Transylvanian Hound Mix                      2
Decker terrier                               1
Transylvanian Hound                          1
Lapponian Herder Mix                         1
Karakachan                                   1
African Village Dog                          1
Slovensky Cuvac Mix                          1
Braque Francais Pyrenean                     1
American English (aka Redtick) Coonhound     1
McNab                                        1
Polish Tatra Mountain Sheepdog               1
Galgo (Spanish Sighthound)                   1
Bavarian Mountain Scent Hound                1
PODENCO                                      1
Norrbottenspets                              1
Prag

## One-Hot-Encode

#### One-Hot-Encode Breed-groups

In [173]:
one_hot_encoded_breed_group = pd.get_dummies(df_HLES['Breed Group'], prefix='breed_group', dummy_na=False)
one_hot_encoded_breed_group

,breed_group_Ancient and Spitz,breed_group_Australian-like,breed_group_Chihuahua,breed_group_Dachshund,breed_group_Golden,breed_group_Herding dogs - Other,breed_group_Hound,breed_group_Labs,breed_group_Mastiff-like Group 1,breed_group_Mastiff-like Group 2,breed_group_Mixed Lab and Golden,breed_group_Mixed Large,breed_group_Mixed Medium,breed_group_Mixed Other,breed_group_Mixed Small,breed_group_Shepherd,breed_group_Spaniels,breed_group_Terriers,breed_group_Toy - Other,breed_group_Working dogs - Non-sport
0,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
3,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33167,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
33168,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
33169,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
33170,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False


In [174]:
id_breed_group_one_hot = pd.concat([df_HLES[['dog_id']], one_hot_encoded_breed_group], axis=1)
id_breed_group_one_hot

,dog_id,breed_group_Ancient and Spitz,breed_group_Australian-like,breed_group_Chihuahua,breed_group_Dachshund,breed_group_Golden,breed_group_Herding dogs - Other,breed_group_Hound,breed_group_Labs,breed_group_Mastiff-like Group 1,...,breed_group_Mixed Lab and Golden,breed_group_Mixed Large,breed_group_Mixed Medium,breed_group_Mixed Other,breed_group_Mixed Small,breed_group_Shepherd,breed_group_Spaniels,breed_group_Terriers,breed_group_Toy - Other,breed_group_Working dogs - Non-sport
0,10509,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,74227,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,32487,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,63150,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
4,33369,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33167,91849,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
33168,33783,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
33169,98862,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
33170,99462,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [192]:
id_breed_group_one_hot.to_pickle('features/breed_group.pkl')
id_breed_group_one_hot.to_csv('features/breed_group.csv', index=False)

#### One-Hot-Encode Breed Types

In [176]:
one_hot_encoded_breed_type = pd.get_dummies(df_HLES['Breed Type'], prefix='breed_type', dummy_na=False)
one_hot_encoded_breed_type

,breed_type_Ancient and Spitz,breed_type_Herding dogs,breed_type_Mastiff-Like,breed_type_Mixed,breed_type_Retriever,breed_type_Scent Hounds,breed_type_Spaniels,breed_type_Terriers,breed_type_Toy dogs,breed_type_Working dogs
0,False,True,False,False,False,False,False,False,False,False
1,False,True,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,True,False
3,False,False,False,True,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
33167,False,False,False,False,False,False,False,True,False,False
33168,False,False,False,False,False,False,False,False,True,False
33169,False,False,True,False,False,False,False,False,False,False
33170,False,False,False,False,False,False,False,False,True,False


In [177]:
id_breed_type_one_hot = pd.concat([df_HLES[['dog_id']], one_hot_encoded_breed_type], axis=1)
id_breed_type_one_hot

,dog_id,breed_type_Ancient and Spitz,breed_type_Herding dogs,breed_type_Mastiff-Like,breed_type_Mixed,breed_type_Retriever,breed_type_Scent Hounds,breed_type_Spaniels,breed_type_Terriers,breed_type_Toy dogs,breed_type_Working dogs
0,10509,False,True,False,False,False,False,False,False,False,False
1,74227,False,True,False,False,False,False,False,False,False,False
2,32487,False,False,False,False,False,False,False,False,True,False
3,63150,False,False,False,True,False,False,False,False,False,False
4,33369,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
33167,91849,False,False,False,False,False,False,False,True,False,False
33168,33783,False,False,False,False,False,False,False,False,True,False
33169,98862,False,False,True,False,False,False,False,False,False,False
33170,99462,False,False,False,False,False,False,False,False,True,False


In [178]:
id_breed_type_one_hot.to_pickle('features/breed_type.pkl')
id_breed_type_one_hot.to_csv('features/breed_type.csv', index=False)

#### One-Hot-Encode Sub Breed

In [179]:
one_hot_encoded_sub_breed = pd.get_dummies(df_HLES['Sub Breed'], prefix='sub_breed', dummy_na=False)
one_hot_encoded_sub_breed

,"sub_breed_""True"" Mastiffs",sub_breed_Ancient and Spitz,sub_breed_Australian,sub_breed_Bulldogs,sub_breed_Chihuahua,"sub_breed_Collie, Corgi, Sheepdog",sub_breed_Dachshund,sub_breed_Golden,sub_breed_Golden mix,sub_breed_Herding-other,...,sub_breed_Setter,sub_breed_Shepherd,sub_breed_Shepherd - Other,sub_breed_Shih Tzu,sub_breed_Sight Hounds,sub_breed_Similar to retrievers,sub_breed_Small,sub_breed_Small Terriers,sub_breed_Sporting,sub_breed_Toy - Other
0,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33167,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
33168,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
33169,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
33170,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [180]:
id_sub_breed_one_hot = pd.concat([df_HLES[['dog_id']], one_hot_encoded_sub_breed], axis=1)
id_sub_breed_one_hot

,dog_id,"sub_breed_""True"" Mastiffs",sub_breed_Ancient and Spitz,sub_breed_Australian,sub_breed_Bulldogs,sub_breed_Chihuahua,"sub_breed_Collie, Corgi, Sheepdog",sub_breed_Dachshund,sub_breed_Golden,sub_breed_Golden mix,...,sub_breed_Setter,sub_breed_Shepherd,sub_breed_Shepherd - Other,sub_breed_Shih Tzu,sub_breed_Sight Hounds,sub_breed_Similar to retrievers,sub_breed_Small,sub_breed_Small Terriers,sub_breed_Sporting,sub_breed_Toy - Other
0,10509,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,74227,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
2,32487,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,63150,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,33369,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33167,91849,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
33168,33783,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
33169,98862,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
33170,99462,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [181]:
id_sub_breed_one_hot.to_pickle('features/sub_breed.pkl')
id_sub_breed_one_hot.to_csv('features/sub_breed.csv', index=False)

#### One-Hot-Encode breeds

In [182]:
one_hot_encoded_breed = pd.get_dummies(df_HLES['dd_breed'], prefix='breed', dummy_na=False)
one_hot_encoded_breed

,breed_Affenpinscher,breed_Affenpinscher Mix,breed_Afghan Hound,breed_African Village Dog,breed_Ainu Dog,breed_Airedale Terrier,breed_Airedale Terrier Mix,breed_Akbash,breed_Akita,breed_Akita Mix,...,breed_Wirehaired Pointing Griffon,breed_Wirehaired Pointing Griffon Mix,breed_Wirehaired Vizsla Mix,breed_Wolf-hybrid Mix,breed_Working Kelpie,breed_Xoloitzcuintli,breed_Xoloitzcuintli Mix,breed_Yorkshire Terrier,breed_Yorkshire Terrier Mix,breed_shiloh shepherd
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33167,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
33168,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
33169,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
33170,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [183]:
id_breed_one_hot = pd.concat([df_HLES[['dog_id']], one_hot_encoded_breed], axis=1)
id_breed_one_hot

,dog_id,breed_Affenpinscher,breed_Affenpinscher Mix,breed_Afghan Hound,breed_African Village Dog,breed_Ainu Dog,breed_Airedale Terrier,breed_Airedale Terrier Mix,breed_Akbash,breed_Akita,...,breed_Wirehaired Pointing Griffon,breed_Wirehaired Pointing Griffon Mix,breed_Wirehaired Vizsla Mix,breed_Wolf-hybrid Mix,breed_Working Kelpie,breed_Xoloitzcuintli,breed_Xoloitzcuintli Mix,breed_Yorkshire Terrier,breed_Yorkshire Terrier Mix,breed_shiloh shepherd
0,10509,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,74227,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,32487,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,63150,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,33369,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33167,91849,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
33168,33783,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
33169,98862,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
33170,99462,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [184]:
id_breed_one_hot.to_pickle('features/breed.pkl')
id_breed_one_hot.to_csv('features/breeds.csv', index=False)

#### One-hot-enconde Top-50 Breeds

In [185]:
# Make a copy of the DataFrame
df_HLES_50 = df_HLES.copy()

# Define the number of most common values to keep
top_n_values = 50

# Identify the 50 most common values in the column
top_values = df_HLES_50['dd_breed'].value_counts().head(top_n_values).index

# Replace values not in the top 50 with NaN
df_HLES_50.loc[~df_HLES_50['dd_breed'].isin(top_values), 'dd_breed'] = np.nan

# Now df_HLES_copy contains the changes, and df_HLES remains unchanged

In [186]:
df_HLES_50.loc[~df_HLES['dd_breed'].isin(top_values), 'dd_breed']

1        NaN
2        NaN
3        NaN
5        NaN
10       NaN
        ... 
33165    NaN
33168    NaN
33169    NaN
33170    NaN
33171    NaN
Name: dd_breed, Length: 10720, dtype: object

In [187]:
df_HLES.loc[df_HLES['dd_breed'].isin(top_values), 'dd_breed']

0            Australian Cattle Dog Mix
4                   Siberian Husky Mix
6        Cavalier King Charles Spaniel
7                         Labrador Mix
8         American Pitbull Terrier Mix
                     ...              
33162                           Beagle
33163             Golden Retriever Mix
33164            Yorkshire Terrier Mix
33166                    Chihuahua Mix
33167            Yorkshire Terrier Mix
Name: dd_breed, Length: 22452, dtype: object

In [188]:
one_hot_encoded_breed_top50 = pd.get_dummies(df_HLES_50['dd_breed'], prefix='breed', dummy_na=False)
one_hot_encoded_breed_top50

,breed_American Pitbull Terrier Mix,breed_American Staffordshire Terrier Mix,breed_Australian Cattle Dog,breed_Australian Cattle Dog Mix,breed_Australian Shepherd,breed_Australian Shepherd Mix,breed_Beagle,breed_Beagle Mix,breed_Border Collie,breed_Border Collie Mix,...,breed_Poodle (Toy) Mix,breed_Poodle Mix,breed_Pug,breed_Shetland Sheepdog,breed_Shih Tzu,breed_Shih Tzu Mix,breed_Siberian Husky,breed_Siberian Husky Mix,breed_Yorkshire Terrier,breed_Yorkshire Terrier Mix
0,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33167,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
33168,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
33169,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
33170,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [189]:
id_breed_one_hot_top50 = pd.concat([df_HLES[['dog_id']], one_hot_encoded_breed_top50], axis=1)
id_breed_one_hot_top50

,dog_id,breed_American Pitbull Terrier Mix,breed_American Staffordshire Terrier Mix,breed_Australian Cattle Dog,breed_Australian Cattle Dog Mix,breed_Australian Shepherd,breed_Australian Shepherd Mix,breed_Beagle,breed_Beagle Mix,breed_Border Collie,...,breed_Poodle (Toy) Mix,breed_Poodle Mix,breed_Pug,breed_Shetland Sheepdog,breed_Shih Tzu,breed_Shih Tzu Mix,breed_Siberian Husky,breed_Siberian Husky Mix,breed_Yorkshire Terrier,breed_Yorkshire Terrier Mix
0,10509,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,74227,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,32487,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,63150,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,33369,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33167,91849,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
33168,33783,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
33169,98862,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
33170,99462,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [190]:
id_breed_one_hot_top50.to_pickle('features/breed_top50.pkl')
id_breed_one_hot_top50.to_csv('features/breed_top50.csv', index=False)

In [191]:
#loaded_df = pd.read_pickle('features/one_hot_encoded_breed_top50_with_id.pkl')
#loaded_df = pd.read_csv('features/one_hot_encoded_breed_top50_with_id.csv')